In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder



In [2]:
import gzip
import json

In [3]:
def reading_in(path):
    thing=[]
    for line in gzip.open(path):
        review_data = json.loads(line)
        subthing = dict()
        for key in review_data:
            subthing[key]= review_data[key]
        thing.append(subthing)
    return thing

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
def pre_pros_y(data):
    encoder = LabelEncoder()
    encoder.fit(data)
    data = encoder.transform(data)
    return data

In [24]:
train = reading_in('../data/classification/music_reviews_train.json.gz')
dev = reading_in('../data/classification/music_reviews_dev.json.gz')

In [25]:
train_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in train]
train_y =  [i['sentiment'] for i in train]

In [26]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, train_size=0.25, random_state=42) #21K

In [27]:
vectorizer = TfidfVectorizer()
tfid = vectorizer.fit(train_x)
train_x = tfid.transform(train_x)
train_x = train_x.toarray()
train_x = np.reshape(train_x, (len(train_x), train_x.shape[1], 1))
train_x = tf.convert_to_tensor(train_x)

In [28]:
train_y = pre_pros_y(train_y)

In [29]:
train_y.shape, train_x.shape

((25000,), TensorShape([25000, 35449, 1]))

In [30]:
tf.keras.backend.clear_session()
tf.keras.backend.learning_phase()

0

In [31]:
model = models.Sequential()
model.add(layers.Conv1D(filters = 32, kernel_size = 10, strides = 5, padding = "same", activation='relu'))
model.add(layers.AveragePooling1D(pool_size = 20, padding = "same", strides = 1))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='sigmoid'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(256, activation='sigmoid'))
model.add(layers.Dense(1, 'sigmoid'))

In [32]:
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])

In [33]:
model.fit(train_x, train_y, epochs=5)

Epoch 1/5
782/782 [==============================] - 201s 256ms/step - loss: 0.6003 - accuracy: 0.6638
Epoch 2/5
782/782 [==============================] - 201s 257ms/step - loss: 0.3603 - accuracy: 0.8432
Epoch 3/5
782/782 [==============================] - 197s 252ms/step - loss: 0.3169 - accuracy: 0.8665
Epoch 4/5
782/782 [==============================] - 196s 250ms/step - loss: 0.2949 - accuracy: 0.8792
Epoch 5/5
782/782 [==============================] - 195s 250ms/step - loss: 0.2776 - accuracy: 0.8886


In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 7090, 32)          352       
                                                                 
 average_pooling1d (AverageP  (None, 7090, 32)         0         
 ooling1D)                                                       
                                                                 
 flatten (Flatten)           (None, 226880)            0         
                                                                 
 dense (Dense)               (None, 256)               58081536  
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                        

In [35]:
dev_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in train]
dev_y =  [i['sentiment'] for i in train]

In [36]:
dev_x = tfid.transform(dev_x[:21000])
dev_x = dev_x.toarray()
dev_x = np.reshape(dev_x, (len(dev_x), dev_x.shape[1], 1))
dev_x = tf.convert_to_tensor(dev_x)

In [37]:
dev_y = pre_pros_y(dev_y[:21000])

In [38]:
model.evaluate(dev_x,dev_y)

657/657 [==============================] - 54s 81ms/step - loss: 0.2894 - accuracy: 0.8841


[0.28939905762672424, 0.8841428756713867]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b88dfe01-c7e1-473c-bcfd-798313fc6522' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>